# Классификация текстов: спам-фильтр для SMS

В этом задании вам предстоит взять открытый датасет с SMS-сообщениями, размеченными на спам ("spam") и не спам ("ham"), построить на нем классификатор текстов на эти два класса, оценить его качество с помощью кросс-валидации, протестировать его работу на отдельных примерах, и посмотреть, что будет происходить с качеством, если менять параметры вашей модели.

In [1]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import MultinomialNB

def write_answer(x, n):
    with open("answer_{}.txt".format(n), "w") as fout:
        fout.write(str(x))

2. Считайте датасет в Python (можете сразу грузить все в память, выборка небольшая), выясните, что используется в качестве разделителей и как проставляются метки классов.

3. Подготовьте для дальнейшей работы два списка: список текстов в порядке их следования в датасете и список соответствующих им меток классов. В качестве метки класса используйте 1 для спама и 0 для "не спама".

In [2]:
data = pd.read_csv('SMSSpamCollection.txt', sep = '\t', names = ['target', 'text'])

In [3]:
data['target'] = data.target.str.replace('spam','1').str.replace('ham','0')
data['target'] = data['target'].astype(int)

In [4]:
data.head()

,target,text
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


4. Используя sklearn.feature_extraction.text.CountVectorizer со стандартными настройками, получите из списка текстов матрицу признаков X.

In [5]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(data.text)
y = data.target.values

5. Оцените качество классификации текстов с помощью LogisticRegression() с параметрами по умолчанию, используя sklearn.cross_validation.cross_val_score и посчитав среднее арифметическое качества на отдельных fold'ах. Установите random_state=2. Параметр cv задайте равным 10. В качестве метрики качества используйте f1-меру. Получившееся качество - один из ответов, которые потребуются при сдаче задания. Ответ округлить до 1 знака после запятой.

In [6]:
clf = LogisticRegression(solver='liblinear') 
clf = LogisticRegression(random_state=2)
cv_score_CVect = cross_val_score(clf, X, y, cv=10, scoring = 'f1').mean()
cv_score_CVect

c:\program files\python37\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.9326402983610631

In [7]:
write_answer(round(cv_score_CVect,1), 1)

6. А теперь обучите классификатор на всей выборке и спрогнозируйте с его помощью класс для следующих сообщений:

"FreeMsg: Txt: CALL to No: 86888 & claim your reward of 3 hours talk time to use from your phone now! Subscribe6GB"

"FreeMsg: Txt: claim your reward of 3 hours talk time"

"Have you visited the last lecture on physics?"

"Have you visited the last lecture on physics? Just buy this book and you will have all materials! Only 99$"

"Only 99$"

Прогнозы классификатора (0 - не спам, 1 - спам), записанные через пробел, будут ответом в одном из вопросов ниже.

In [9]:
import numpy as np
new_sms = ["FreeMsg: Txt: CALL to No: 86888 & claim your reward of 3 hours talk time to use from your phone now! Subscribe6GB",
           "FreeMsg: Txt: claim your reward of 3 hours talk time",
           "Have you visited the last lecture on physics?",
           "Have you visited the last lecture on physics? Just buy this book and you will have all materials! Only 99$",
           "Only 99$"]
X_test = vectorizer.transform(new_sms)

In [11]:
clf.fit(X,y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=2, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [12]:
prediction = clf.predict(X_test)
prediction

array([1, 1, 0, 0, 0])

In [13]:
write_answer(' '.join([str(i) for i in prediction]), 2)

7. Задайте в CountVectorizer параметр ngram_range=(2,2), затем ngram_range=(3,3), затем ngram_range=(1,3). Во всех трех случаях измерьте получившееся в кросс-валидации значение f1-меры, округлите до второго знака после точки, и выпишете результаты через пробел в том же порядке. В данном эксперименте мы пробовали добавлять в признаки n-граммы для разных диапазонов n - только биграммы, только триграммы, и, наконец, все вместе - униграммы, биграммы и триграммы. Обратите внимание, что статистики по биграммам и триграммам намного меньше, поэтому классификатор только на них работает хуже. В то же время это не ухудшает результат сколько-нибудь существенно, если добавлять их вместе с униграммами, т.к. за счет регуляризации линейный классификатор не склонен сильно переобучаться на этих признаках.

In [14]:
clf = LogisticRegression(solver='liblinear') 
cv_scores_LR = []
for nr in [(2,2), (3,3), (1,3)]:
    vect = CountVectorizer(ngram_range=nr)
    X_nr = vect.fit_transform(data.text)
    cv_score = cross_val_score(clf, X_nr, y, cv=10, scoring = 'f1').mean()
    cv_scores_LR.append(round(cv_score,2))
    
cv_scores_LR

[0.82, 0.73, 0.93]

In [15]:
write_answer(' '.join([str(i) for i in cv_scores_LR]), 3)

8. Повторите аналогичный п.7 эксперимент, используя вместо логистической регрессии MultinomialNB(). Обратите внимание, насколько сильнее (по сравнению с линейным классификатором) наивный Байес страдает от нехватки статистики по биграммам и триграммам.

По какой-то причине обучение наивного байесовского классификатора через Pipeline происходит с ошибкой. Чтобы получить правильный ответ, отдельно посчитайте частоты слов и обучите классификатор.

In [16]:
clf = MultinomialNB() 
cv_scores_NB = []
for nr in [(2,2), (3,3), (1,3)]:
    vect = CountVectorizer(ngram_range=nr)
    X_nr = vect.fit_transform(data.text)
    cv_score = cross_val_score(clf, X_nr, y, cv=10, scoring = 'f1').mean()
    cv_scores_NB.append(round(cv_score,2))
cv_scores_NB

[0.65, 0.38, 0.89]

In [17]:
write_answer(' '.join([str(i) for i in cv_scores_NB]), 4)

9. Попробуйте использовать в логистической регрессии в качестве признаков Tf\*idf из TfidfVectorizer на униграммах. Повысилось или понизилось качество на кросс-валидации по сравнению с CountVectorizer на униграммах? (напишите в файле с ответом 1, если повысилось, -1, если понизилось, и 0, если изменилось не более чем на 0.01). Обратите внимание, что результат перехода к tf*idf не всегда будет таким - если вы наблюдаете какое-то явление на одном датасете, не надо сразу же его обобщать на любые данные.

In [18]:
clf = LogisticRegression(solver='liblinear') 

vect = TfidfVectorizer()
X_tfidf = vect.fit_transform(data.text)
cv_score_tfidf = cross_val_score(clf, X_tfidf, y, cv=10, scoring = 'f1').mean()

In [19]:
print('cross-val score CountVestorizer:',cv_score_CVect)
print('cross-val score TfidfVestorizer:',cv_score_tfidf)

cross-val score CountVestorizer: 0.9326402983610631
cross-val score TfidfVestorizer: 0.8528599554172456


In [20]:
res = 1 if cv_score_CVect<cv_score_tfidf else -1
res

-1

In [21]:
write_answer(res, 5)